In [1]:
import numpy as np
import pandas as pd
import os
import re

/dsi/sbm/OrrBavly/kidney_data/original_samples/

Load original TSV files, create df containing grouped by aminoAcid seqs with count column

In [2]:
def downsampling(rep, n):
    read_vec = rep['readCount'].values
    read_vec = [int(val) for val in read_vec]
    read_indices = np.zeros(np.sum(read_vec), dtype=int)
    dummy = 0
    for i in range(len(read_vec)):
        read_indices[dummy:dummy + read_vec[i]] = i
        dummy += read_vec[i]
    new_counts = np.random.choice(read_indices, n, replace=False)
    new_reads = np.zeros(len(read_vec), dtype=int)
    for idx in new_counts:
            new_reads[idx] += 1
    downsampled_data = rep.copy()
    downsampled_data['readCount'] = new_reads
    downsampled_data = downsampled_data[downsampled_data["readCount"]!= 0].reset_index(drop = True)
    downsampled_data['readFraction'] = downsampled_data['readCount'] / downsampled_data['readCount'].sum()
    return downsampled_data

In [5]:
# Directory containing sample files
samples_dir = "/dsi/sbm/OrrBavly/kidney_data/original_samples"  # Update with actual path
sample_files = [f for f in os.listdir(samples_dir) if f.endswith('.tsv')]
sample_dfs = []

for sample_file in sample_files:
        file_path = os.path.join(samples_dir, sample_file)

        # Read and filter the sample file
        data = pd.read_csv(file_path, sep='\t')
        filtered_data = data.dropna(subset=['aminoAcid'])

        # Group by 'aminoAcid' and aggregate the counts
        grouped_data = (
            filtered_data.groupby('aminoAcid')['count (templates/reads)']
            .sum()
            .reset_index()
        )
        grouped_data.columns = ['aminoAcid', 'readCount']
        # downsample data
        # downsampled_df = downsampling()

        # Append the grouped data with a sample identifier
        grouped_data['Sample'] = sample_file.split('.')[0]
        

        # Append individual sample DataFrame to the list
        sample_dfs.append(grouped_data)

/tmp/ipykernel_2863653/1688682434.py:10: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t')
/tmp/ipykernel_2863653/1688682434.py:10: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t')
/tmp/ipykernel_2863653/1688682434.py:10: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t')
/tmp/ipykernel_2863653/1688682434.py:10: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t')
/tmp/ipykernel_2863653/1688682434.py:10: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t')
/tmp/ipykernel_2863653/1688682434.py:10: DtypeWarning: Columns (24,25) have mixe

In [8]:
def summarize_sample_totals(sample_dfs):
    sample_sums = []

    for df in sample_dfs:
        sample_name = df['Sample'].iloc[0]  # Extract sample name from DataFrame
        total_sum = df['readCount'].sum()
        sample_sums.append({'Sample': sample_name, 'Total Sum': total_sum})

    # Convert to DataFrame and sort in descending order
    summary_df = pd.DataFrame(sample_sums).sort_values(by='Total Sum', ascending=False)

    # Print nicely formatted summary
    print("\nSample Totals in Descending Order:")
    print(summary_df)

    return summary_df

In [10]:
# Summarize and print sample totals
summarized_totals = summarize_sample_totals(sample_dfs)

# # Output individual DataFrames
# for i, df in enumerate(sample_dfs):
#     print(f"\nDataFrame for Sample {i + 1}:")
#     print(df)


Sample Totals in Descending Order:
              Sample  Total Sum
54    AR_073_D0_TCRB     479528
55   STA_062_D0_TCRB     397656
60    AR_056_D0_TCRB     355492
38    AR_055_D0_TCRB     350609
105  STA_030_D0_TCRB     344752
..               ...        ...
161   AR_029_D0_TCRB        570
84    AR_030_D0_TCRB        569
64    AR_032_D0_TCRB        562
173   AR_026_D0_TCRB        534
93    AR_034_D0_TCRB        503

[189 rows x 2 columns]


## Run downsample

In [11]:
counts_threshold = 19789
downsampled_folder = '/dsi/sbm/OrrBavly/kidney_data/downsamples_19789'

In [16]:

counter = 0
for sample_df in sample_dfs:
    if sample_df['readCount'].sum() >= counts_threshold:
        downsampled_df = downsampling(sample_df, counts_threshold)
        sample_name = sample_df['Sample'].iloc[0]
        shortened_name = f"{sample_name.split('_')[0]}_{sample_name.split('_')[1]}.csv"
        full_path = os.path.join(downsampled_folder, shortened_name)
        # Drop the 'Sample' column before saving
        downsampled_df = downsampled_df.drop(columns=['Sample'])
        downsampled_df.to_csv(full_path, index=False)
        counter += 1
    else:
        print(f"skipping file: {sample_df['Sample'][0]}")
print(counter)

skipping file: STA_073_D0_TCRB
skipping file: AR_009_D0_TCRB
skipping file: AR_024_D0_TCRB
skipping file: AR_037_D0_TCRB
skipping file: STA_002_D0_TCRB
skipping file: STA_049_D0_TCRB
skipping file: STA_007_D0_TCRB
skipping file: AR_019_D0_TCRB
skipping file: AR_058_D0_TCRB
skipping file: AR_069_D0_TCRB
skipping file: AR_086_D0_TCRB
skipping file: AR_031_D0_TCRB
skipping file: AR_032_D0_TCRB
skipping file: AR_036_D0_TCRB
skipping file: STA_025_D0_TCRB
skipping file: AR_061_D0_TCRB
skipping file: AR_046_D0_TCRB
skipping file: AR_049_D0_TCRB
skipping file: AR_030_D0_TCRB
skipping file: AR_034_D0_TCRB
skipping file: STA_066_D0_TCRB
skipping file: AR_028_D0_TCRB
skipping file: AR_033_D0_TCRB
skipping file: AR_064_D0_TCRB
skipping file: AR_059_D0_TCRB
skipping file: AR_018_D0_TCRB
skipping file: STA_040_D0_TCRB
skipping file: AR_025_D0_TCRB
skipping file: AR_020_D0_TCRB
skipping file: AR_029_D0_TCRB
skipping file: STA_046_D0_TCRB
skipping file: AR_026_D0_TCRB
skipping file: STA_047_D0_TCRB
1

## Create single Data csv file

In [ ]:
# Function to create a unified TCR repertoire matrix
def create_tcr_matrix(csv_files, input_dir, output_file):
    all_tcrs = set()
    sample_data = {}

    # Step 1: Collect all unique TCR sequences and fraction counts for each sample
    for csv_file in csv_files:
        file_path = os.path.join(input_dir, csv_file)
        data = pd.read_csv(file_path)

        # Ensure required columns exist
        if {'aminoAcid', 'readFraction'}.issubset(data.columns):
            sample_name = os.path.splitext(csv_file)[0]
            sample_data[sample_name] = {
                row['aminoAcid']: row['readFraction']
                for _, row in data.iterrows()
            }
            all_tcrs.update(data['aminoAcid'].dropna())
        else:
            print(f"Skipping file {csv_file} as required columns are missing.")

    # Step 2: Create a DataFrame where rows are samples and columns are TCR sequences
    all_tcrs = sorted(all_tcrs)  # Sort TCR sequences for consistent column ordering
    tcr_matrix = pd.DataFrame(columns=['Sample'] + all_tcrs)

    for sample_name, tcrs in sample_data.items():
        row = {'Sample': sample_name}
        for tcr in all_tcrs:
            row[tcr] = tcrs.get(tcr, 0)  # Fill with readFraction or 0 if missing
        tcr_matrix = tcr_matrix.append(row, ignore_index=True)

    # Step 3: Save the unified matrix to a CSV file
    tcr_matrix.to_csv(output_file, index=False)
    print(f"Unified TCR matrix saved to {output_file}")

In [ ]:
# Directory containing input CSV files
input_dir = "/dsi/sbm/OrrBavly/kidney_data/downsamples_19789/"  # Update with actual path
csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

# Output file path
output_file = "unified_tcr_matrix.csv"  # Update with desired output path

# Create the TCR matrix
create_tcr_matrix(csv_files, input_dir, output_file)
